In [22]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import EvalPrediction
from setfit import SetFitModel, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
import wandb
import pandas as pd
import torch
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

### Save the path to the different datasets

In [3]:
train_en_path = "./data_sources/train/train_en.csv"
test_en_path = "./data_sources/test/test_en.csv"

train_it_path = "./data_sources/train/train_it.csv"
test_it_path = "./data_sources/test/test_it.csv"

train_es_path = "./data_sources/train/train_es.csv"
test_es_path = "./data_sources/test/test_es.csv"

### Set up W&B

In [4]:
wandb.login()

wandb: Currently logged in as: sravisconti (sravisconti-projects) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Load data in DatasetDict

In [13]:
# Load CSV manually for the train split
train_df = pd.read_csv(train_it_path)
test_df = pd.read_csv(test_it_path)

# Convert back to Hugging Face Datasets
dataset_it = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df)
})

### Define Metrics

In [11]:
def compute_metrics(preds, labels):
    precision_macro = precision_score(labels, preds, average="macro", zero_division=0)
    recall_macro = recall_score(labels, preds, average="macro", zero_division=0)
    f1_macro = f1_score(labels, preds, average="macro", zero_division=0)

    return {
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "macro_f1": f1_macro,
    }


### Train and Evaluate

In [15]:
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SetFitModel.from_pretrained(model_name)

# to match labels with meaning: 0 --> "offensive", 1 --> "reappropriative"
model.labels = ["offensive", "reappropriative"]

args = TrainingArguments(
    batch_size=8,
    num_epochs=2,
    num_iterations=5,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_it["train"],
    metric=compute_metrics,
    column_mapping={"text": "text", "label": "label"}
)

wandb.init(project="multi-pride-setfit-pipeline", name="setfit-it")

trainer.train()
final_results = trainer.evaluate(dataset_it["test"])
wandb.log(final_results)
print("Final test metrics:", final_results)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to the training dataset
Map: 100%|██████████| 868/868 [00:00<00:00, 11473.43 examples/s]


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


macro_f1,▁
precision_macro,▁
recall_macro,▁
train/embedding_loss,▆▅▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▃▂▁▁▁█▄▂▁▁▁▇
train/epoch,▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▄▁▁▂▂▃▄▅▆█▁▂▅▆▁
train/global_step,▁▁▁▂▂▂▃▃▃▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇██▁▁▁▂▂▂▂▃▂▂▂▂▃
train/grad_norm,▄▃▃▁▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▂▁▁▁▁▁▆▆▂▁▁▁▁█
train/learning_rate,▁▁▁▁▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▇▇▇▇▇▇█▇▆▆▄▃▃▃▂▅█▄▃▁
macro_f1,0.87399
precision_macro,0.85735
recall_macro,0.89448


c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\wandb\analytics\sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


***** Running training *****
  Num unique pairs = 8680
  Batch size = 8
  Num epochs = 2
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,0.280100
50,0.178600
100,0.137500
150,0.111500
200,0.098400
250,0.067200
300,0.060600
350,0.043700
400,0.019500
450,0.006800


c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Applying column mapping to the evaluation dataset
***** Running evaluation *****


Final test metrics: {'precision_macro': 0.8738624873609707, 'recall_macro': 0.9001623376623377, 'macro_f1': 0.8861024033437827}


### Metric Report

In [32]:
# Mapping label strings -> integers
label2id = {label: i for i, label in enumerate(model.labels)}

# Get raw predictions from SetFit (strings)
raw_preds = trainer.model.predict(dataset_it["test"]["text"])

# Convert to integers
y_pred = np.array([label2id[p] for p in raw_preds])

y_true = np.array(dataset_it["test"]["label"])

print("Classification Report:")
print(classification_report(
    y_true,
    y_pred,
    target_names=model.labels
))

print("Confusion Matrix:")

cm_df = pd.DataFrame(
    confusion_matrix(y_true, y_pred),
    index=[f"True: {label}" for label in model.labels],
    columns=[f"Pred: {label}" for label in model.labels]
)
print(cm_df)

Classification Report:
                 precision    recall  f1-score   support

      offensive       0.97      0.94      0.95       176
reappropriative       0.78      0.86      0.82        42

       accuracy                           0.93       218
      macro avg       0.87      0.90      0.89       218
   weighted avg       0.93      0.93      0.93       218

Confusion Matrix:
                       Pred: offensive  Pred: reappropriative
True: offensive                    166                     10
True: reappropriative                6                     36


### Error Analysis

In [33]:
# Indices of false positives : predicted offensive but actually reappropriate
fn_indices = np.where((y_true == 1) & (y_pred == 0))[0]

fn_sentences = [dataset_it["test"][i]['text'] for i in fn_indices]

print("Predicted offensive but actually reappropriate:\n")
for i, sentence in enumerate(fn_sentences):
    print(i)
    print(sentence)
    print("\n")

# save to CSV with space between them
pd.DataFrame(fn_sentences, columns=["sentence"]).to_csv("error_analysis/set_fit_pipeline/false_negatives_it.csv", index=True, sep='\t')

Predicted offensive but actually reappropriate:

0
Che branco di guardoni morbosi, “fruga-mutande” della sessualità altrui, della vita altrui, degli amori altrui, c’è al mondo. Mi dispiace, @USER 🤍

“Ancor prima di ammetterlo a me stesso, io ero un ricchione, e questo andava scritto, ripetuto, urlato - soprattutto sussurrato”


1
chi pensa che il problema sia l'omofobia è IL problema. voglio però dire frocio quando mi pare solo perché mi piace la parola. FROCIO. sono omofobo? lol l'unica cosa che mi spaventa è il vaccino ago bua. voglio urlarlo: VIVA I FROCI, lunga vita a loro, quindi a me


2
@USER @USER Per qualcuno sono gay,per altri transgender, per latri ancora LGBT. Per so froci e rottinculo! Bloccatemi vigliacchi di Twitter


3
@USER non sono fiduciosa negli spazi di aggregazione online. Letteralmente ci sono i queer frocialisti eccetera eccetera e poi i circoletti di gay che postano nudes ogni giorno hanno onlyfans e sono convinti di vivere in un porno 24/7

è un incubo


4
@US

In [34]:
# Indices of false positives : predicted reappropriate but actually offensive
fp_indices = np.where((y_true == 0) & (y_pred == 1))[0]

fp_sentences = [dataset_it["test"][i]['text'] for i in fp_indices]

print("Predicted reappropriate but actually offensive:\n")
for i, sentence in enumerate(fp_sentences):
    print(i)
    print(sentence)
    print("\n")

# save to CSV with space between them
pd.DataFrame(fp_sentences, columns=["sentence"]).to_csv("error_analysis/set_fit_pipeline/false_positives_it.csv", index=True, sep='\t')

Predicted reappropriate but actually offensive:

0
Tutti i vostri "frocio", mi aprivano un varco verso le camere a gas. Mi hanno ucciso ma IO HO POTUTO LOTTARE, loro no. Anche per questo serve una legge contro l'omofobia

#omocausto #GiornataDellaMemoria2021 @USER


1
Lo ripeto: i froci fanno vomitare e i loro pride di merda altrettanto visto che la domenica vorrei dormire e mi svegliano con Lady Gaga a palla


2
Ieri sono uscito con una tuta arcobaleno e un tipo mi fa "frocio se vuoi ti faccio conoscere qualcuno che te lo mette al culo" e all'inizio non l'ho presa benissimo


3
Per inciso: il “frocio di m.” era il paziente che stava operando e già in anestesia totale #omofobia #7luglio

“Guardate se devo operare sto frocio di m...”: sospeso primario nel Varesotto URL


4
#Morisi se sei finocchio di dx non sei garantito dal ddl zan mentre se sei di sx e fai il porco nei gaypride o spacci a cielo aperto e tutto regolare


5
Ma lultimo che "froci" e subito dopo "LGbt 4ever e bla bla bla 